# **IMPORT THƯ VIỆN**

In [1]:
# 📘 02_training.ipynb
# Nhiệm vụ: Huấn luyện mô hình ResNet1D + Attention cho ECG 12 leads

import os, sys
sys.path.append("../")

import numpy as np
import torch
from torch.utils.data import DataLoader

from training.dataset_loader import ECGDataset
from training.model import ResNet1DAttention
from training.model_inceptiontime import InceptionTime1D
from training.train import train_model
from training.evaluate import evaluate_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Thiết bị sử dụng:", device)

Thiết bị sử dụng: cuda


# **ĐỌC DỮ LIỆU CHIA TRAIN/VAL/TEST**

In [2]:
# === 1️⃣ Đọc dữ liệu chia train/val/test ===
data_dir = r"E:\NCKH - 2026\ECG Project\data\processed\splits"

X_train = np.load(os.path.join(data_dir, "train_files.npy"), allow_pickle=True)
X_val = np.load(os.path.join(data_dir, "val_files.npy"), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, "test_files.npy"), allow_pickle=True)

y_train = np.load(os.path.join(data_dir, "y_train.npy"))
y_val = np.load(os.path.join(data_dir, "y_val.npy"))
y_test = np.load(os.path.join(data_dir, "y_test.npy"))

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
print(f"Số lớp nhãn: {y_train.shape[1]}")

Train: 31393 | Val: 4485 | Test: 8970
Số lớp nhãn: 8


# **TẠO DATASET - DATA LOADER**

In [3]:
# === 2️⃣ Tạo Dataset & DataLoader ===
train_dataset = ECGDataset(X_train, y_train, augment=True)
val_dataset = ECGDataset(X_val, y_val)
test_dataset = ECGDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print("✅ DataLoader đã sẵn sàng")

✅ DataLoader đã sẵn sàng


# **TRAIN DỮ LIỆU**

# **RESNET18**

In [4]:
# === 3️⃣ Khởi tạo và huấn luyện mô hình ===
num_classes = y_train.shape[1]
model = ResNet1DAttention(num_classes=num_classes)
print(model)

model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    y_train=y_train,
    num_epochs=25,
    patience=5,
    lr=1e-3,
    device=device
)

# Lưu mô hình tốt nhất
model_path = r"E:\NCKH - 2026\ECG Project\models\resnet1d_attention_best.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
print(f"✅ Đã lưu mô hình vào: {model_path}")

ResNet1DAttention(
  (layer1): BasicBlock1D(
    (conv1): Conv1d(12, 64, kernel_size=(3,), stride=(2,), padding=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Conv1d(12, 64, kernel_size=(1,), stride=(2,))
  )
  (layer2): BasicBlock1D(
    (conv1): Conv1d(64, 128, kernel_size=(3,), stride=(2,), padding=(1,))
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Conv1d(64, 128, kernel_size=(1,), stride=(2,))
  )
  (layer3): BasicBlock1D(
    (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(2,), padding=(1,))
    (bn1): BatchNorm1d(128, eps=1e

In [5]:
# === 4️⃣ Đánh giá mô hình trên test set ===
preds_bin, preds, cms = evaluate_model(model, test_loader, y_test, device=device)


Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3257
           1       0.49      0.87      0.62      1623
           2       0.68      0.97      0.80      1615
           3       0.79      0.94      0.86      1451
           4       0.16      0.66      0.26       492
           5       0.18      0.97      0.30       360
           6       0.19      0.87      0.32       335
           7       0.33      0.98      0.49       308

   micro avg       0.52      0.91      0.66      9441
   macro avg       0.47      0.90      0.57      9441
weighted avg       0.67      0.91      0.74      9441
 samples avg       0.59      0.89      0.68      9441


Confusion Matrices:
Label 0:
[[5393  320]
 [ 254 3003]]

Label 1:
[[5867 1480]
 [ 213 1410]]

Label 2:
[[6633  722]
 [  45 1570]]

Label 3:
[[7163  356]
 [  84 1367]]

Label 4:
[[6746 1732]
 [ 165  327]]

Label 5:
[[6979 1631]
 [  10  350]]

Label 6:
[[7414 1221]
 [  43

C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [6]:
# === 5️⃣ Thử dự đoán một mẫu ECG ===
model.eval()
sample_signal, true_label = test_dataset[0]
sample_signal = sample_signal.unsqueeze(0).to(device)

with torch.no_grad():
    output = torch.sigmoid(model(sample_signal)).cpu().numpy().flatten()

print("⚙️ Dự đoán:")
print("Giá trị xác suất:", np.round(output, 3))
print("Nhãn nhị phân:", (output > 0.5).astype(int))
print("Nhãn thật:", true_label.numpy().astype(int))

⚙️ Dự đoán:
Giá trị xác suất: [0.001 0.888 0.01  0.525 0.39  0.005 0.001 0.   ]
Nhãn nhị phân: [0 1 0 1 0 0 0 0]
Nhãn thật: [0 1 0 0 0 0 0 0]


# **INCEPTION TIME 34**

In [4]:
# === Huấn luyện mô hình InceptionTime1D + Attention ===
num_classes = y_train.shape[1]
model = InceptionTime1D(in_channels=12, num_classes=num_classes)
print(model)

model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    y_train=y_train,
    num_epochs=15,
    patience=5,
    lr=1e-3,
    device=device
)

# === Lưu mô hình tốt nhất ===
model_path = r"E:\NCKH - 2026\ECG Project\models\inceptiontime1d_attention_best.pth"
os.makedirs(os.path.dirname(model_path), exist_ok=True)
torch.save(model.state_dict(), model_path)
print(f"Đã lưu mô hình InceptionTime1D vào: {model_path}")


InceptionTime1D(
  (block1): InceptionResidualBlock1D(
    (inception): InceptionBlock1D(
      (bottleneck): Conv1d(12, 16, kernel_size=(1,), stride=(1,), bias=False)
      (conv1): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (conv2): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv3): Conv1d(16, 16, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
      (pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_pool): Conv1d(12, 16, kernel_size=(1,), stride=(1,), bias=False)
      (norm): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (relu): ReLU()
    )
    (residual): Sequential(
      (0): Conv1d(12, 64, kernel_size=(1,), stride=(1,), bias=False)
      (1): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (relu): ReLU()
  )
  (block2): InceptionResidualBlock1D(
    (inception): Ince

Epoch 1: train_loss=0.8601, val_loss=0.6819


Epoch 2: train_loss=0.6579, val_loss=0.5954


Epoch 3: train_loss=0.5820, val_loss=0.5916


Epoch 4: train_loss=0.5374, val_loss=0.6472


...

Epoch 11: train_loss=0.4233, val_loss=0.4809


Epoch 12: train_loss=0.4154, val_loss=0.4748


Epoch 13: train_loss=0.4039, val_loss=0.4652


Epoch 14: train_loss=0.3996, val_loss=0.4795


Epoch 15: train_loss=0.3945, val_loss=0.4500
Đã lưu mô hình InceptionTime1D vào: E:\NCKH - 2026\ECG Project\models\inceptiontime1d_attention_best.pth


In [5]:
# === Đánh giá mô hình trên test set ===
preds_bin, preds, cms = evaluate_model(model, test_loader, y_test, device=device)


Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      3257
           1       0.77      0.87      0.82      1623
           2       0.68      0.97      0.80      1615
           3       0.85      0.95      0.90      1451
           4       0.20      0.66      0.31       492
           5       0.17      0.98      0.29       360
           6       0.18      0.86      0.30       335
           7       0.25      1.00      0.40       308

   micro avg       0.55      0.94      0.70      9441
   macro avg       0.50      0.91      0.59      9441
weighted avg       0.72      0.94      0.79      9441
 samples avg       0.66      0.91      0.73      9441


Confusion Matrices:
Label 0:
[[5324  389]
 [  59 3198]]

Label 1:
[[6919  428]
 [ 208 1415]]

Label 2:
[[6607  748]
 [  55 1560]]

Label 3:
[[7266  253]
 [  67 1384]]

Label 4:
[[7187 1291]
 [ 167  325]]

Label 5:
[[6849 1761]
 [   6  354]]

Label 6:
[[7317 1318]
 [  46

C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

In [4]:
num_classes = y_train.shape[1]  # hoặc điền số class thủ công, ví dụ 5
model = InceptionTime1D(in_channels=12, num_classes=num_classes)

In [9]:
model_path = r"E:\NCKH - 2026\ECG Project\models\inceptiontime1d_attention_best.pth"

# Load vào model
model.load_state_dict(torch.load(model_path, map_location="cuda" if torch.cuda.is_available() else "cpu"))
model.to(device)
model.eval()  # chuyển sang chế độ suy luận
print("Đã load lại mô hình thành công!")

Đã load lại mô hình thành công!


C:\Users\ASUS\AppData\Local\Temp\ipykernel_33088\1361336070.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location="cu

# **TÌM NGƯỠNG TỐT NHẤT**

In [10]:
from misc.threshold_finder import find_best_threshold

In [11]:
from sklearn.metrics import f1_score, classification_report

In [12]:
model.eval()
y_val_true, y_val_pred_proba = [], []

with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        logits = model(X_batch)
        probs = torch.sigmoid(logits).cpu().numpy()  # multi-label => sigmoid
        y_val_pred_proba.append(probs)
        y_val_true.append(y_batch.cpu().numpy())

y_val_true = np.vstack(y_val_true)
y_val_pred_proba = np.vstack(y_val_pred_proba)

In [13]:
best_thresholds = find_best_threshold(y_val_true, y_val_pred_proba)
print("Best thresholds per class:", np.round(best_thresholds, 3))

Best thresholds per class: [0.65 0.55 0.65 0.85 0.65 0.8  0.9  0.9 ]


In [14]:
# --- Áp dụng threshold ---
y_val_pred = (y_val_pred_proba > best_thresholds).astype(int)

# --- Tính F1 trung bình ---
macro_f1 = f1_score(y_val_true, y_val_pred, average="macro")
print(f"Validation Macro-F1 = {macro_f1:.4f}\n")

# --- In báo cáo chi tiết ---
target_names = [
    "Nhịp chậm xoang", "Nhịp xoang bình thường", "Cuồng nhĩ", "Chênh lên đoạn ST", "Loạn nhịp xoang",
    "Rung nhĩ", "Chênh xuống đoạn ST", "Trục điện tim lệch trái"
]
report = classification_report(y_val_true, y_val_pred, target_names=target_names, zero_division=0)
print("Classification Report:\n")
print(report)

Validation Macro-F1 = 0.6464

Classification Report:

                         precision    recall  f1-score   support

        Nhịp chậm xoang       0.91      0.98      0.95      1632
 Nhịp xoang bình thường       0.78      0.84      0.81       803
              Cuồng nhĩ       0.71      0.92      0.80       813
      Chênh lên đoạn ST       0.95      0.91      0.93       741
        Loạn nhịp xoang       0.31      0.45      0.37       249
               Rung nhĩ       0.20      0.91      0.32       176
    Chênh xuống đoạn ST       0.41      0.55      0.47       182
Trục điện tim lệch trái       0.36      0.95      0.52       154

              micro avg       0.68      0.89      0.77      4750
              macro avg       0.58      0.82      0.65      4750
           weighted avg       0.77      0.89      0.81      4750
            samples avg       0.74      0.87      0.78      4750



In [14]:
save_dir = "../models"
os.makedirs(save_dir, exist_ok=True)

torch.save(model.state_dict(), f"{save_dir}/resnet3_best.pth")
np.save(f"{save_dir}/best_thresholds_r3.npy", best_thresholds)

print("Saved model and thresholds!")

Saved model and thresholds!
